## Importing Libraries

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
from datasets import load_dataset

# wandb
import wandb

## Login to Hugging Face

In [2]:
load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token,  # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


In [3]:
model_name = "Waktaverse-Llama-2-KO-7B-Instruct"  # ADD YOUR MODEL NAME HERE
username = "PathFinderKR"  # ADD YOUR USERNAME HERE
repo_id = f"{username}/{model_name}"  # repository id

## Login to Weights & Biases

In [4]:
api_key = os.getenv("WANDB_API_KEY")
wandb.login(
    key=api_key  # ADD YOUR API KEY HERE
)
wandb.init(project=model_name)

wandb: Currently logged in as: pathfinderkr. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/pathfinder/.netrc


## Device

In [5]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


In [6]:
# Flash Attention Implementation
if device == "cuda:0":
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float32
print(f"Attention Implementation = {attn_implementation}")

Attention Implementation = flash_attention_2


## Hyperparameters

In [7]:
################################################################################
# Tokenizer parameters
################################################################################
max_length=4096
padding="do_not_pad"  # "max_length", "longest", "do_not_pad"
truncation=True

################################################################################
# Generation parameters
################################################################################
num_return_sequences=1
max_new_tokens=1024
do_sample=True  # True for sampling, False for greedy decoding
temperature=0.6
top_p=0.9
repetition_penalty=1.1

################################################################################
# bitsandbytes parameters
################################################################################
load_in_4bit=True
bnb_4bit_compute_dtype=torch_dtype
bnb_4bit_quant_type="nf4"  # "nf4", #fp4"
bnb_4bit_use_double_quant=True

################################################################################
# LoRA parameters
################################################################################
task_type="CAUSAL_LM"
target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
r=16
lora_alpha=32
lora_dropout=0.1
bias="none"

################################################################################
# Training parameters
################################################################################
output_dir="./results"
logging_dir="./logs"
save_strategy="epoch"
logging_strategy="steps"  # "steps", "epoch"
if logging_strategy == "steps":
    logging_steps=10
else:
    logging_steps=None
save_total_limit=1
report_to="wandb"

num_train_epochs=1
per_device_train_batch_size=2
gradient_accumulation_steps=2
gradient_checkpointing=True
bf16=True
learning_rate=2e-5
lr_scheduler_type="cosine"  # "constant", "linear", "cosine"
warmup_ratio=0.1
optim = "paged_adamw_32bit"
weight_decay=0.1

################################################################################
# SFT parameters
################################################################################
max_seq_length=4096
packing=True

## Tokenizer

In [8]:
# Korean Tokenizer ID
tokenizer_id = "meta-llama/Llama-2-7b-chat-hf"

In [9]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

In [10]:
# Add padding token
tokenizer.pad_token = tokenizer.eos_token

In [11]:
# Vocabulary size
print(f"Vocabulary size: {len(tokenizer)}")
# Special tokens
print(f"Special tokens: {tokenizer.special_tokens_map}")
# Padding side
print(f"Padding side: {tokenizer.padding_side}")

Vocabulary size: 32000
Special tokens: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}
Padding side: right


In [12]:
# Display the tokenizer configuration
display(Markdown(f'```{tokenizer}```'))

```LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}```

## Model

In [13]:
# Model ID for base model
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [14]:
# Quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

In [15]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
    quantization_config=quantization_config,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)```

In [17]:
# Number of parameters
print(f"Number of parameters (in billions): {model.num_parameters() / 1e9:.2f}")

Number of parameters (in billions): 6.74


## Dataset

In [18]:
# Dataset ID
dataset_id = "beomi/KoAlpaca-v1.1a"

In [19]:
# Load the dataset
dataset = load_dataset(dataset_id)

In [20]:
# Dataset information
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [21]:
# Dataset example
print(dataset["train"][0]["instruction"])
print(dataset["train"][0]["output"])

양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.


## Preprocessing

In [22]:
# Shuffle the dataset
dataset = dataset.shuffle(seed=42)

In [23]:
# Alpaca dataset format: 
# {"instruction": [str],
#   "input": [str],
#   "output": [str]}

# Korean
def prompt_without_input(example):
    text = (
        "<s>[INST]<<SYS>>\n"
        "다음은 작업을 설명하는 지시사항입니다. 요청을 적절하게 완료하는 응답을 작성하세요.\n"
        "<</SYS>>\n\n"

        f"{example['instruction']}[/INST]{example['output']}</s>"
        )
    return {'text': text}

# Apply the alpaca prompt to the dataset
dataset = dataset.map(prompt_without_input)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [24]:
# Display the first example
print(dataset["train"][0]["text"])

<s>[INST]<<SYS>>
다음은 작업을 설명하는 지시사항입니다. 요청을 적절하게 완료하는 응답을 작성하세요.
<</SYS>>

케첩과 마요네즈의 입구 모양은 왜 다른가요? 
보통 케찹과 마요네즈는 한 세트자나요? 그런데 보면 케찹은 원터치 뚜껑에다가 새 케찹은 뚜껑을 돌려서 열면 얇은 알미늄 같은 막도 이꾸또 케찹이 나오는 구멍이 동그랗자나요? 근데 마요네즈는 돌리는 뚜껑에다가 처음에 새로 사도 알미늄 같은 막두 엄꾸나오는 구멍 모양도 별 모양인데 왜 그런가요?[/INST]케첩과 마요네즈의 입구 모양이 다른 것은 끈적이는 정도 즉, 물질의 점성(Viscosity) 차이 때문입니다. 마요네즈는 케첩보다 점성이 높기 때문에 돌리는 뚜껑에 별 모양의 입구를 만들어도 내부 액체의 모양이 유지됩니다. 하지만 케첩은 끈적거림이 적기 때문에 별 모양의 입구를 만들어도 케첩 내부의 액체가 곧 둥글게 뭉쳐지기 때문에 케첩의 뚜껑은 동그란 구멍입니다. 따라서 입구 모양은 물질의 점성(Viscosity) 차이 때문입니다.</s>


## Supervised Fine-Tuning (LoRA)

In [25]:
# Prepare model for kbit training
model = prepare_model_for_kbit_training(model)

In [26]:
lora_config = LoraConfig(
    task_type=task_type,
    target_modules=target_modules,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias=bias
)

In [27]:
# Number of trainable parameters
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 39,976,960 || all params: 6,778,392,576 || trainable%: 0.5898


In [28]:
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    save_strategy=save_strategy,
    logging_strategy=logging_strategy,
    logging_steps=logging_steps,
    save_total_limit=save_total_limit,
    report_to=report_to,
    
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    bf16=bf16,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    optim=optim,
    weight_decay=weight_decay
)

In [29]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    peft_config=lora_config,
    args=training_args,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=packing
)

Generating train split: 0 examples [00:00, ? examples/s]

In [30]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass us

Step,Training Loss
10,1.429800
20,1.406800
30,1.331900
40,1.306400
50,1.225900
60,1.141300
70,1.050600
80,0.989300
90,0.939000
100,0.915600


TrainOutput(global_step=946, training_loss=0.8208524938869678, metrics={'train_runtime': 17803.4844, 'train_samples_per_second': 0.213, 'train_steps_per_second': 0.053, 'total_flos': 6.181714590003364e+17, 'train_loss': 0.8208524938869678, 'epoch': 1.0})

In [31]:
wandb.finish()
trainer.save_model(model_name)

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,██▆▂▁▁▁▁▂▃▃▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
train/learning_rate,▂▃▅▇███████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train/loss,█▇▆▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,6.181714590003364e+17
train/epoch,1.0
train/global_step,946
train/grad_norm,0.32686
train/learning_rate,0.0
train/loss,0.7454


/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Inference

In [32]:
def prompt_template(system, user):
    return (
        "<s>[INST]<<SYS>>\n"
        f"{system}\n"
        "<</SYS>>\n\n"
        
        f"{user}[/INST]"
    )

In [33]:
def generate_response(system ,user):
    prompt = prompt_template(system, user)
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [34]:
#system_prompt = "You are a helpful assistant. Respond to the following user prompt."
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."
#user_prompt = "Write me a poem about Machine Learning."
user_prompt = "머신러닝에 대한 시를 써주세요."

In [35]:
response = generate_response(system_prompt, user_prompt)
print(response)

/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> [INST]<<SYS>>
다음 지시사항에 대한 응답을 작성해주세요.
<</SYS>>

머신러닝에 대한 시를 써주세요.[/INST]시:

처음에는 어려웠지만, 계속해서 연습하면서 자연스럽게 배운 머신러닝이라는 말은 독일어의 "Maschinen"에서 유래된 단어입니다. 이 단어는 기계나 동물의 무거운 일을 불가능하게 하는 능력을 뜻하고, 따라서 머신러닝은 쉽게 풀리지 않는 문제를 풀 수 있는 기술을 말합니다. 이 말은 과학 분야에서도 사용되며, 복잡한 계산 방식을 빠르게 완료하는 기술을 말합니다.</s>
